In [1]:
import json

In [2]:
import json
pid_dict = {}
# Define the path to your log file
log_file_path = './auditbeat_output.ndjson'
detection_point = []
records = []
# Open the log file
with open(log_file_path, 'r') as file:
    for line in file:
        try:
            # Parse each line as JSON
            log_entry = json.loads(line)   
            records.append(log_entry)         
            # Check if the log entry contains 'malicious program'
            if 'secret.txt' in json.dumps(log_entry) and "auditd" in log_entry: 
                detection_point.append(log_entry)
            
        except json.JSONDecodeError:
            # Skip lines that are not valid JSON
            continue

In [ ]:
# backward track
interpret = {}
while len(detection_point) != 0:
    i = detection_point.pop(0)
    parent = i["process"].get("parent").get("pid")
    pid = i["process"].get("pid")
    syscall = i["auditd"]["data"].get("syscall")
    paths = i.get("auditd", {}).get("paths", [])
    primary_object = []
    if len(paths) > 0:
        for path in paths:
            primary_object.append(path.get("name", None))
    for j in records:
        if "process" in j and j["process"].get("pid") == parent:
            detection_point.append(j)
    if pid in interpret:
        interpret[pid][0].append(syscall)
        interpret[pid][1].append(primary_object)
    else:
        interpret[pid] =  ([syscall], [primary_object], parent)
    
    # Append the tuple to the list


In [ ]:
# find root pid = 2354
interpret

{144871: (['openat'], [['/home/student/secret/secret.txt']], 144870),
 144870: (['execve',
   'setresuid',
   'openat',
   'openat',
   'setresuid',
   'setuid',
   'setgid'],
  [['/usr/bin/sudo', '/lib64/ld-linux-x86-64.so.2'],
   [],
   ['/etc/sudoers'],
   ['/etc/shadow'],
   [],
   [],
   []],
  144797),
 144797: (['execve',
   'execve',
   'execve',
   'execve',
   'execve',
   'execve',
   'execve'],
  [['./run-attack.sh', '/bin/bash', '/lib64/ld-linux-x86-64.so.2'],
   ['./run-attack.sh', '/bin/bash', '/lib64/ld-linux-x86-64.so.2'],
   ['./run-attack.sh', '/bin/bash', '/lib64/ld-linux-x86-64.so.2'],
   ['./run-attack.sh', '/bin/bash', '/lib64/ld-linux-x86-64.so.2'],
   ['./run-attack.sh', '/bin/bash', '/lib64/ld-linux-x86-64.so.2'],
   ['./run-attack.sh', '/bin/bash', '/lib64/ld-linux-x86-64.so.2'],
   ['./run-attack.sh', '/bin/bash', '/lib64/ld-linux-x86-64.so.2']],
  2354)}

In [ ]:
# construct pid tree 
pid_tree = {}
for record in reversed(records):
    if "process" in record:
        pid = record["process"].get("pid")
        parent = record["process"].get("parent").get("pid")
        child_list = pid_tree.get(parent, set())
        child_list.add(pid)
        pid_tree[parent] = child_list
# forward track from the root
target_pids = set()
pid_stack = [2354]

while len(pid_stack) != 0:
    target_pid = pid_stack.pop(0)
    if target_pid in pid_tree:
        children = pid_tree[target_pid]
        target_pids = target_pids.union(children) 
        pid_stack.extend(list(children))

In [10]:
target_pids

{144797,
 144798,
 144799,
 144802,
 144803,
 144805,
 144806,
 144808,
 144809,
 144811,
 144812,
 144814,
 144815,
 144817,
 144818,
 144820,
 144821,
 144832,
 144834,
 144837,
 144838,
 144840,
 144841,
 144843,
 144844,
 144846,
 144847,
 144848,
 144849,
 144850,
 144851,
 144852,
 144853,
 144854,
 144855,
 144856,
 144857,
 144858,
 144859,
 144860,
 144861,
 144863,
 144864,
 144865,
 144867,
 144868,
 144869,
 144870,
 144871,
 144873,
 144875}